In [33]:
import pandas as pd
import tensorflow as tf
import numpy as np

In [34]:
xy = pd.read_csv('supermarket_sales - Sheet1.csv') 

In [35]:
xy.head() 

,Invoice ID,Branch,City,Customer type,Gender,Product line,Unit price,Quantity,Tax 5%,Total,Date,Time,Payment,cogs,gross margin percentage,gross income,Rating
0,750-67-8428,A,Yangon,Member,Female,Health and beauty,74.69,7,26.1415,548.9715,1/5/2019,13:08,Ewallet,522.83,4.761905,26.1415,9.1
1,226-31-3081,C,Naypyitaw,Normal,Female,Electronic accessories,15.28,5,3.8200,80.2200,3/8/2019,10:29,Cash,76.40,4.761905,3.8200,9.6
2,631-41-3108,A,Yangon,Normal,Male,Home and lifestyle,46.33,7,16.2155,340.5255,3/3/2019,13:23,Credit card,324.31,4.761905,16.2155,7.4
3,123-19-1176,A,Yangon,Member,Male,Health and beauty,58.22,8,23.2880,489.0480,1/27/2019,20:33,Ewallet,465.76,4.761905,23.2880,8.4
4,373-73-7910,A,Yangon,Normal,Male,Sports and travel,86.31,7,30.2085,634.3785,2/8/2019,10:37,Ewallet,604.17,4.761905,30.2085,5.3


In [36]:
xy.drop('Invoice ID', axis=1, inplace=True)                       # 불필요한 항목 제거
xy.drop('Branch', axis=1, inplace=True)
xy.drop('City', axis=1, inplace=True)
xy.drop('Product line', axis=1, inplace=True)
xy.drop('Unit price', axis=1, inplace=True)
xy.drop('Tax 5%', axis=1, inplace=True)
xy.drop('Total', axis=1, inplace=True)
xy.drop('Date', axis=1, inplace=True)
xy.drop('Time', axis=1, inplace=True)
xy.drop('cogs', axis=1, inplace=True)
xy.drop('gross margin percentage', axis=1, inplace=True)
xy.drop('gross income', axis=1, inplace=True)
xy.drop('Rating', axis=1, inplace=True)

In [37]:
# 자료를 숫자로 바꾸기 위한 과정
customer_mapping={"Member":0, "Normal":1}                          # 회원인 경우는 0, 비회원은 1
gender_mapping={"Female":0, "Male":1}                              # 여자는 0, 남자는 1
payment_mapping={"Ewallet":0, "Cash":1, "Credit card":2}           # Ewallet 0, Cash 1, Credit card 2
xy['Customer type'] = xy['Customer type'].map(customer_mapping)    # 문자를 순자로 변환
xy['Gender'] = xy['Gender'].map(gender_mapping)
xy['Payment'] = xy['Payment'].map(payment_mapping)
xy.head()

,Customer type,Gender,Quantity,Payment
0,1,0,7,0
1,0,0,5,1
2,0,1,7,2
3,1,1,8,0
4,0,1,7,0


In [38]:
xy.info()   # Dtype 확인

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype
---  ------         --------------  -----
 0   Customer type  1000 non-null   int64
 1   Gender         1000 non-null   int64
 2   Quantity       1000 non-null   int64
 3   Payment        1000 non-null   int64
dtypes: int64(4)
memory usage: 31.4 KB


In [39]:
XY_A = np.array(xy, dtype='float32')       # float32로 형변환 및 데이터를 배열로 가져옴
x_data = XY_A[:, 1:4]  # gender, quantity, payment
y_data = XY_A[:, [0]]  # customer type

X = tf.placeholder(tf.float32, shape=[None, 3])
Y = tf.placeholder(tf.float32, shape=[None, 1])

W = tf.Variable(tf.random_normal([3, 1]),  name='weight') # tf.random_normal([x,y])-x:들어오는 값 갯수/y:나가는 값 갯수
b = tf.Variable(tf.random_normal([1]), name='bias') # tf.random_normal([y])-y:나가는 값 갯수

hypothesis = tf.sigmoid(tf.matmul(X,W) + b) # tf.matmul-matrix 곱하기

cost = -tf.reduce_mean(Y * tf.log(hypothesis) + (1 - Y) * tf.log(1 - hypothesis))
train = tf.train.GradientDescentOptimizer(learning_rate=0.01).minimize(cost)

predicted = tf.cast(hypothesis > 0.5, dtype=tf.float32) # hypothesis > 0.5의 결과값(True, False)은 cast를 통해 0 또는 1로 출력
accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted, Y), dtype=tf.float32)) 
# tf.equal()을 통해 예측값과 Y값 비교, cast를 통해 0 또는 1로 출력

In [40]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    feed = {X:x_data, Y:y_data}
    for step in range(10001):
        sess.run(train, feed_dict=feed)
        if step % 200 == 0:
            print(step, sess.run(cost, feed_dict=feed))
            
    h, c, a = sess.run([hypothesis, predicted, accuracy],
                      feed_dict=feed)
    print("\nHypothesis: ", h, "\nCorrect (Y): ", c, "\nAccuracy: ", a)

0 1.6955252
200 0.7532196
400 0.7455917
600 0.73928785
800 0.7339055
1000 0.72921735
1200 0.7250853
1400 0.72141784
1600 0.7181493
1800 0.715228
2000 0.7126135
2200 0.7102706
2400 0.70817
2600 0.7062858
2800 0.70459557
3000 0.70307904
3200 0.70171845
3400 0.70049775
3600 0.6994028
3800 0.69842076
4000 0.69754016
4200 0.69675076
4400 0.6960428
4600 0.69540846
4800 0.69483995
5000 0.6943307
5200 0.6938744
5400 0.69346577
5600 0.6930996
5800 0.69277173
6000 0.6924784
6200 0.69221544
6400 0.69198024
6600 0.69176966
6800 0.6915811
7000 0.6914123
7200 0.69126123
7400 0.691126
7600 0.691005
7800 0.69089675
8000 0.6908
8200 0.6907133
8400 0.69063556
8600 0.6905663
8800 0.690504
9000 0.6904484
9200 0.6903988
9400 0.6903541
9600 0.69031435
9800 0.6902787
10000 0.6902468

Hypothesis:  [[0.49568444]
 [0.5310872 ]
 [0.5152426 ]
 [0.4448232 ]
 [0.444901  ]
 [0.444901  ]
 [0.4957632 ]
 [0.49544817]
 [0.5662571 ]
 [0.56617975]
 [0.49592072]
 [0.48020843]
 [0.49584195]
 [0.4446676 ]
 [0.5306949 ]
 [0.5